<a href="https://colab.research.google.com/github/Data-Sinence-course-Winter-2020/Assignments/blob/master/2_Vectorizer_and_Test_and_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#V8 - Clean code 2.0 *
import nltk
import random
import numpy as np 
import pandas as pd 
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from random import seed
from random import randint
from random import shuffle
import array
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nltk.corpus.gutenberg.fileids() #define list of possible books from gutenberg
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt',
'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt',
'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt',
'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt',
'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt',
'shakespeare-macbeth.txt', 'whitman-leaves.txt']

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [0]:
#choosing 7 books in list in lower case
#origin is the origin veriosn of chosen books
origin = [nltk.corpus.gutenberg.raw('austen-emma.txt'), nltk.corpus.gutenberg.raw('milton-paradise.txt'), nltk.corpus.gutenberg.raw('chesterton-ball.txt') , nltk.corpus.gutenberg.raw('melville-moby_dick.txt') , nltk.corpus.gutenberg.raw('edgeworth-parents.txt') , nltk.corpus.gutenberg.raw('bryant-stories.txt') , nltk.corpus.gutenberg.raw('whitman-leaves.txt') ]
books = [nltk.corpus.gutenberg.raw('austen-emma.txt').lower(), nltk.corpus.gutenberg.raw('milton-paradise.txt').lower(), nltk.corpus.gutenberg.raw('chesterton-ball.txt').lower() , nltk.corpus.gutenberg.raw('melville-moby_dick.txt').lower() , nltk.corpus.gutenberg.raw('edgeworth-parents.txt').lower() , nltk.corpus.gutenberg.raw('bryant-stories.txt').lower() , nltk.corpus.gutenberg.raw('whitman-leaves.txt').lower() ]

In [0]:
#extract words from string without punctuation
def book_punc(i):
  books[i] = re.sub('['+string.punctuation+']', '', str(books[i])).split() 


In [0]:
#removing stop words 
def book_stopwords(i):
  tokenized_words = books[i] 
  stop_words = stopwords.words('english')
  books[i]=[word for word in tokenized_words if word not in stop_words]

In [0]:
#Stemmer 
def book_stemmer(i):
  ps = PorterStemmer()
  example_words = books[i]
  new = []
  for w in example_words:
   new.append(ps.stem(w))
  books[i]=new

In [0]:
#number of words in each document
def book_docWords(i):
  documentW=len(books[i]) // 200   #number of words per documents
  return int(documentW)

In [0]:
#make 200 random numbers as a pointer to each word of a document. The goal is start reading from the pointer as a random number 
def book_randNum (documentWo,i):
  documentW = documentWo
  value = []
  #Preventing possible errors if each section of a book had less than 150 words
  if (documentW >= 150):
    tguess = documentW - 150
    for i in range (200): #should be  200
      tmp=randint(0,tguess)
      value.append(tmp)
  else:
    tguess = documentW
    for i in range (200): #should be 200
      tmp=randint(0,tguess)
      if (i == 199):  # should be 200 - 1
        tmp=tmp-documentW
      value.append(tmp)
  return value

In [0]:
#make a list of 200 sublist *  contains 150 words (for each book, obviously!)
def book_listWord (documentWs ,values , i):
  value = values
  documentW = documentWs
  
  dataset = []
  datasetF = []
  listoflists = []
  a_list = []
  lstF = []

  res=books[i]
  for i in range (200):  # after testing should be 200 ~ number of sample documents for each book
    point = (i * documentW) + value[i]
    for x in range (150): #should be 150 ~ number of words in each sample
      dataset.append(res[point])
      point += 1
      #print(dataset) #check each line
      if len(dataset) == 150 :  #should be 150 ~ appending words to the dataset
        datasetF.append((list(dataset)))
        dataset.clear()      
        
  return list (datasetF)


In [0]:
#MAKE DATA FRAME FROM FINAL DATASET (which a list containst 7 list , 200 sublist and each sublist contains 150 words) ~ this data is for the whole books 

def book_lable(finList):
  lst = finList
  dct = {}
  booknamelist = ['Emma', 'Paradise', 'Ball', 'Moby', 'Parents' , 'Stories' ,  'Leaves']
  authornamelist = ['Austun', 'Milton', 'Chesterton', 'Melville', 'Edgeworth' , 'Briant' , 'Whitman']
  column_names = ["BookName" , "AuthorName" , "Content"]
  df2 = pd.DataFrame(columns = column_names)

  for j in range (7): #always be 7 (7 list)
    for k in range (200):  #should be 200 after tests
      datas = { 
      "BookName": booknamelist[j] , "AuthorName": authornamelist[j] , "Content": (finList[j][k ]) #from finlist[j][k] to finlist[j][k+1]
         }
      df = pd.DataFrame.from_dict(datas)
      frames = [df]
      result = pd.concat(frames)
      df2 = df2.append(result)
      
  #df2=' '.join(df2)
  return df2

In [12]:
#The main function
bookGlist = []
bookFlist = []
values = []

column_names = ["BookName" , "AuthorName" , "Content"]
dataFrame_final = pd.DataFrame(columns = column_names)
for i in range(7):
  book_punc(i)
  book_stopwords(i)
  book_stemmer(i)
  documentW = book_docWords(i)
  values = book_randNum (documentW , i)
  bookGlist = book_listWord(documentW , values , i)
  bookFlist.append(bookGlist)


print("DONE :: End of books process")
#print(bookFlist) #Lists ins order without lable (Just for TEST)

dataFrame_final = book_lable(bookFlist)  #labling books
#dataFrame_final = dataFrame_final.reset_index() #make index - for the shuffling
#dataFrame_finalshuff = dataFrame_final.reindex(np.random.permutation(dataFrame_final.index))  #This is the final target which has shuffled
print (dataFrame_final)
#dataFrame_finalshuff[['BookName', 'AuthorName' , 'Content']]

#print("list of shuffled Dataaform")
#print (dataFrame_final[['BookName', 'AuthorName' , 'Content']]) #dataFrame without shuffling for test
#print (dataFrame_finalshuff[['BookName', 'AuthorName' , 'Content']])
#print (dataFrame_finalshuff[['BookName', 'AuthorName' , 'Content']])
#>>> dataFrame_finalshuff can be implemented for the rest of the code <<<
#print(bookGlist)
#type(dataFrame_finalshuff[['BookName', 'AuthorName' , 'Content']])


DONE :: End of books process
    BookName AuthorName   Content
0       Emma     Austun     earli
1       Emma     Austun    period
2       Emma     Austun    mother
3       Emma     Austun       die
4       Emma     Austun      long
..       ...        ...       ...
145   Leaves    Whitman     sleep
146   Leaves    Whitman   billion
147   Leaves    Whitman   billion
148   Leaves    Whitman  trillion
149   Leaves    Whitman  trillion

[210000 rows x 3 columns]


In [13]:
#Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(dataFrame_final.Content).toarray()
labels = dataFrame_final.AuthorName
#print (labels) 
features.shape
#print(type (features.shape)) #<class 'tuple'>

(210000, 4972)

In [0]:
#Naive
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(dataFrame_final['Content'], dataFrame_final['AuthorName'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [15]:
print(clf.predict(count_vect.transform(["khoda"])))

['Chesterton']


In [18]:
#Test
dataFrame_final[dataFrame_final['Content'] == "love"]

,BookName,AuthorName,Content
57,Emma,Austun,love
87,Emma,Austun,love
35,Emma,Austun,love
41,Emma,Austun,love
107,Emma,Austun,love
...,...,...,...
3,Leaves,Whitman,love
114,Leaves,Whitman,love
38,Leaves,Whitman,love
92,Leaves,Whitman,love
